# Example from the Tutorial

This notebook executes the example from the Introductory Tutorial

In [12]:
import pystra as pr
import numpy as np

## Establish the model

In [13]:
def example_limitstatefunction(r, X1, X2, X3):
    """
    example limit state function
    """
    return r - X2 * (1000 * X3) ** (-1) - (X1 * (200 * X3) ** (-1)) ** 2

Define limit state function

In [14]:
limit_state = pr.LimitState(example_limitstatefunction)

Set some options - here we suppress the console output

In [15]:
options = pr.AnalysisOptions()
options.setPrintOutput(False)

Create the stochastic model object

In [16]:
stochastic_model = pr.StochasticModel()

Define and add random variables to the stochastic model object

In [17]:
stochastic_model.addVariable(pr.Lognormal("X1", 500, 100))
stochastic_model.addVariable(pr.Normal("X2", 2000, 400))
stochastic_model.addVariable(pr.Uniform("X3", 5, 0.5))

Define constants that can be part of the stochastic model for ease of argument passing

In [18]:
stochastic_model.addVariable(pr.Constant("r", 1))

If the random variables are correlated, then define a correlation matrix, otherwise no correlatin matrix is needed

In [19]:
stochastic_model.setCorrelation(
    pr.CorrelationMatrix([[1.0, 0.3, 0.2], [0.3, 1.0, 0.2], [0.2, 0.2, 1.0]])
)

## FORM Analysis

In [20]:
form = pr.Form(
    analysis_options=options,
    stochastic_model=stochastic_model,
    limit_state=limit_state,
)
form.run()

Show detailed output

In [21]:
form.showDetailedOutput()


FORM
Pf              	 3.9717297753e-02
BetaHL          	 1.7539761407
Model Evaluations 	 164
------------------------------------------------------
Variable   	    U_star 	       X_star 	     alpha
X1         	  1.278045 	   631.504135 	 +0.728414
X2         	  0.407819 	  2310.352495 	 +0.232354
X3         	 -1.129920 	     4.517374 	 -0.644534
r          	       --- 	     1.000000 	       ---



## SORM Analysis
For efficiency, we can pass the FORM results object if it exists, otherwise it will be called automatically.

In [22]:
sorm = pr.Sorm(
    analysis_options=options,
    stochastic_model=stochastic_model,
    limit_state=limit_state,
    form=form,
)
sorm.run()



RESULTS FROM RUNNING SECOND ORDER RELIABILITY METHOD

Generalized reliability index:  1.848997968430195
Probability of failure:         0.03222905303518171

Curavture 1: -0.0414313089629377
Curavture 2: 0.36356407339249475



Detailed output

In [23]:
sorm.showDetailedOutput()


FORM/SORM
Pf FORM         		 3.9717297753e-02
Pf SORM Breitung 		 3.2229053035e-02
Pf SORM Breitung HR 	 3.1158626160e-02
Beta_HL         		 1.7539761407
Beta_G Breitung 		 1.8489979684
Beta_G Breitung HR 		 1.8640317035
Model Evaluations 		 180
------------------------------------------------------
Curvature 1: -0.0414313089629377
Curvature 2: 0.36356407339249475
------------------------------------------------------
Variable   	    U_star 	       X_star 	     alpha
X1         	  1.278045 	   631.504135 	 +0.728414
X2         	  0.407819 	  2310.352495 	 +0.232354
X3         	 -1.129920 	     4.517374 	 -0.644534
r          	       --- 	     1.000000 	       ---



## Distribution Analysis

In [24]:
da = pr.DistributionAnalysis(
    analysis_options=options,
    stochastic_model=stochastic_model,
    limit_state=limit_state,
)
da.run()

## Crude Monte Carlo Simulation

In [25]:
cmc = pr.CrudeMonteCarlo(
    analysis_options=options,
    stochastic_model=stochastic_model,
    limit_state=limit_state,
)
cmc.run()

## Importance Sampling

In [26]:
ismc = pr.ImportanceSampling(
    analysis_options=options,
    stochastic_model=stochastic_model,
    limit_state=limit_state,
)
ismc.run()

## Results

In [28]:
beta = form.getBeta()
failure = form.getFailure()

print(f"Beta is {beta}, corresponding to a failure probability of {failure}")

Beta is 1.7539761407409615, corresponding to a failure probability of [0.0397173]
